In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import pulp
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import joblib

# Load and preprocess training data
train_data = pd.read_csv('Datathon Dataset.csv')

# Clean invalid dates and machinery types
valid_machineries = ['Backhoe Loader', 'Excavators(crawler)', 'Loaders (Wheeled)', 
                     'Skid Steer Loaders', 'Compactors', 'Tele Handlers']
train_data = train_data[train_data['Infrastructure_Machineries'].isin(valid_machineries)]
train_data['Date'] = pd.to_datetime(train_data['Date'], errors='coerce')
train_data = train_data.dropna(subset=['Date'])

# Feature engineering
train_data['year'] = train_data['Date'].dt.year
train_data['month'] = train_data['Date'].dt.month
train_data['day'] = train_data['Date'].dt.day
train_data['dayofweek'] = train_data['Date'].dt.dayofweek

# Encode categorical variables
train_data = pd.get_dummies(train_data, columns=['Infrastructure_Machineries', 'Region'])

# Prepare features and target
X = train_data.drop(columns=['Daily_Sales_Quantity', 'Unnamed: 0', 'Un_Named', 'Customer_Id', 'Date'])
y = train_data['Daily_Sales_Quantity']

# Train XGBoost model
model = XGBRegressor()
model.fit(X, y)

# New data for prediction
new_data = pd.DataFrame({
    'Date': ['1/1/2025'],
    'Daily_Sales _Percentage': [0.034463806],
    'Market_Share': [35],
    'Political': [1],
    'Marketing': [1],
    'Budget': [5000.56],
    'Infrastructure_Machineries': ['Loaders (Wheeled)'],
    'Region': ['Sherrichester']
})

# Process new data similarly
new_data['Date'] = pd.to_datetime(new_data['Date'], format='%d/%m/%Y')
new_data['year'] = new_data['Date'].dt.year
new_data['month'] = new_data['Date'].dt.month
new_data['day'] = new_data['Date'].dt.day
new_data['dayofweek'] = new_data['Date'].dt.dayofweek

# Ensure all columns match training data
new_data_encoded = pd.get_dummies(new_data, columns=['Infrastructure_Machineries', 'Region'])
for col in X.columns:
    if col not in new_data_encoded.columns:
        new_data_encoded[col] = 0
new_data_encoded = new_data_encoded[X.columns]

# Predict sales quantity
predicted_quantity = model.predict(new_data_encoded)

print(f'Predicted Sales Quantity: {predicted_quantity[0]}')



# Save model to disk
joblib.dump(model, 'xgboost_model.pkl')


C:\Users\my pc\AppData\Local\Temp\ipykernel_15080\3143748689.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data_encoded[col] = 0
C:\Users\my pc\AppData\Local\Temp\ipykernel_15080\3143748689.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data_encoded[col] = 0
C:\Users\my pc\AppData\Local\Temp\ipykernel_15080\3143748689.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once us

Predicted Sales Quantity: 70.2639389038086


['xgboost_model.pkl']